In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pax import units
%matplotlib inline

In [ ]:
from pax.configuration import load_configuration
from pax.dsputils import adc_to_pe
pax_config = load_configuration('XENON1T')["DEFAULT"]
to_pe = np.array([adc_to_pe(pax_config, ch) 
                  for ch in range(pax_config['n_channels'])])

In [ ]:
import strax
fn = 'tl_uncomp'
records = strax.load(fn)
strax.sort_by_time(records)
strax.baseline(records)
hits = strax.find_hits(records)
strax.cut_outside_hits(records, hits)

In [ ]:
# Find peaks at single-electron granularity
peaks = strax.find_peaks(hits, gap_threshold=250, min_hits=12)

In [ ]:
# Group these together in larger peaks (separated by no more than 1 ms)
tails = strax.find_peaks(peaks, 
                         gap_threshold=int(1e6), 
                         min_hits=1,
                         max_duration=int(1 * units.ms),
                         dtype=strax.peak_dtype(n_channels=260,
                                                n_sum_wv_samples=100))

In [ ]:
# TODO: crash when I do this on peaks?

In [ ]:
strax.sum_waveform(tails, records, to_pe)
tails['area'] = (tails['area_per_channel'] * to_pe.reshape(1, -1)).sum(axis=1)

In [ ]:
np.sort(tails['area'])[-10:]

In [ ]:
big_i = np.argsort(tails['area'])[-3]
p = tails[big_i]
w = p['sum_waveform']
dt = 10 * units.ns * p['downsample_factor']
t = np.arange(0, len(w)) * dt
plt.plot(t / units.ms, w, linestyle='steps-mid')
plt.title('%d PE' % peaks[big_i]['area'])
plt.ylim(0, None)
plt.xlim(0, None)
plt.yscale('symlog')
plt.xlabel("Time (ms)")
plt.ylabel("Amplitude PE/bin")
plt.plot(t / units.ms, 1e4 * np.exp(-t/(400 * units.us)), marker='.', color='red')
plt.xlim(0, 1)
#plt.xscale('symlog')

# OLD

In [ ]:
from multihist import Hist1d

In [ ]:
def peakfinder(gap_threshold=300, min_hits=3):
    peaks = strax.find_peaks(hits, gap_threshold=gap_threshold, min_hits=min_hits)
    print(thr, len(records), len(hits), len(peaks))
    
    
    return peaks

In [ ]:
for thr in (250, 300,  350, 400, 500):
    peaks = peakfinder(thr)
    Hist1d(peaks['area'], np.linspace(0, 50, 50)).plot(label=thr)
plt.legend()

In [ ]:
peaks = peakfinder(gap_threshold=int(100 * 1e3), min_hits=100)

In [ ]:
np.sort(peaks['area'])[-10:]

In [ ]:
big_i = np.argsort(peaks['area'])[-1]
p = peaks[big_i]
p['area']

In [ ]:
(p['endtime'] - p['time']) / 1000

In [ ]:
w = p['sum_waveform']
dt = 10 * units.ns * p['downsample_factor']
t = np.arange(0, len(w)) * dt
plt.plot(t / units.ms, w, linestyle='steps-mid')
plt.title('%d PE' % peaks[big_i]['area'])
plt.ylim(0, None)
plt.xlim(0, None)
plt.yscale('symlog')
plt.xlabel("Time (ms)")
plt.ylabel("Amplitude PE/bin")
#plt.xlim(0, 1)
#plt.xscale('symlog')

In [ ]:
t0 = peaks[big_i]['time']
ps = peaks[big_i:]
amps = ps['area'] / (ps['endtime'] - ps['time'])
plt.scatter((ps['time'] - t0) / units.us, 
            , 
            marker='.')
plt.ylim(1e-3, 1)
plt.yscale('log')
plt.xscale('log')
plt.xlim(1, 1e6)

In [ ]:
big_i = np.argsort(peaks['area'])[-2]
t0 = peaks[big_i]['time']
for i in range(big_i, big_i + 100):
    p = peaks[i]
    w = p['sum_waveform']
    dt = 10 * units.ns * p['downsample_factor']
    t = np.arange(0, len(w)) * dt
    t += p['time'] - t0
    plt.plot(t / units.ms, w, linestyle='steps-mid')
plt.title('%d PE' % peaks[big_i]['area'])
plt.ylim(0, None)
plt.xlim(0, None)
plt.yscale('symlog')
plt.xlabel("Time (ms)")
plt.ylabel("Amplitude PE/bin")
#plt.xscale('symlog')

In [ ]:
for i in range(big_i, big_i + 20):